# Arquitecturas avanzadas de NLP. Sequence 2 Sequence

[Uno de los mejores resources](https://distill.pub/2016/augmented-rnns/)

Hoy/ahora, veremos una de las arquitecturas más usadas, para un tipo de problema muy concreto, pero muy extendido. Si os acordais de Language Modeling, generábamos secuencias, con una palabra o carácter de semilla, pero podía ir a cualquier parte la secuencia. Con Sequence 2 Sequence (Seq2Seq - S2S), lo que queremos es condicionar el output de nuestra arquitectura a todo un contexto.

![Seq2Seq](https://cdn-images-1.medium.com/max/1600/1*_6-EVV3RJXD5KDjdnxztzg@2x.png)

Esta arquitectura apareció por primera vez en 2014, [paper](https://www.aclweb.org/anthology/D14-1179), y aunque tuvo mucha repercusión, no fue hasta que en 2015... lo vemos despues!

Estas arquitecturas, ya en 2014 mostraron gran potencial, y se confirmó cuando empresas como Google, decidieron cambiar productos tan importantes como el Google Translate a arquitecturas similares a esta. Os dejó un blog [post](https://codesachin.wordpress.com/2017/01/18/understanding-the-new-google-translate/) para que veais, y el [paper](https://arxiv.org/pdf/1609.08144.pdf) original dónde se explica el funcionamiento de la arquitectura. Otra aplicación bastante interesante, es la de generar respuestas a correos electrónicos de forma automática [info aquí](https://ai.googleblog.com/2015/11/computer-respond-to-this-email.html).

### Vale bien, pero que hace esto?

El dibujo de arriba, si seguis el timeline (time-steps), queda bastante claro. Primero codificamos un mensaje (steps-1-4), luego inicializamos el decoder (flecha entre 4 y 5), y empezamos la decodificación (5-7). 

Durante el entreno, se pasan dos inputs distintos, el input del encoder, y el del decoder. El input del encoder aquí sería




In [0]:
input_encoder = ["<SOS>", "how", "are", "you", "?", "<EOS>"]
input_decoder = ["<SOS>", "I", "am", "good", "<EOS>"]
target_decoder = input_decoder[1:]
target_decoder

['I', 'am', 'good', '<EOS>']

Fijaros que el encoder no tiene un target, sino que calcularemos la loss en el decoder, y pasaremos el gradiente hacía atrás desde el decoder, hasta el primer step del encoder.

Del dibujo de arriba, ya lo habeis visto absolutamente todo. Las celdas azules verdes y azules son...? Normalmente son celdas RNN, ya sean vanilla RNN, LSTMs o GRUs (que aunque no las hemos introducido en el curso, son una variamente algo más "barata" que las LSTMs).

La única flecha que os debería preocupar es la de verde a azul. En esa parte estamos copiando el último estado de nuestra encoder-RNN, y usandolo como estado inicial en el decoder-RNN. Si os acordáis del dibujo, de las LSTMs, básicamente queremos h_t, cuando t = T, es decir en su último estado.

![](https://i.imgur.com/nE53oh1.png)

### Keras tip

![](https://i.imgur.com/V9Pkm8d.png)

Keras nos permite obtener el estado de una LSTM, seteando return_state a True.

Esto modifica el output, devolviendonos 3 tensores, el output, el hidden state, i el cell state. En caso normal, el output i el hidden state, son lo mismo, pero esto puede ser modificado con el keyword return_sequences, que nos permite obtener una matriz con un output por timestep, el hidden state (del último estado) y el cell_state del último estado.

Ahora que ya sabemos como recoger el hidden state, ya podemos ir a implementar un Seq2Seq en Keras.

No implementaremos nada en numpy, porque ya tenemos casi todo implementado (almenos el forward pass) en sesiones pasadas, con lo que saltaremos a Keras directamente.


## Imports

In [0]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.layers import TimeDistributed
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import numpy as np
from random import shuffle, choice, sample

import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

Using TensorFlow backend.


## Data

In [0]:
def generate_dummy_data():
    x = [ix for ix in range(100)]
    data = []
    for ix_source in range(3, 5):
        for ix_target in range(3, 5):
            for ix, _ in enumerate(x):
                data.append((x[ix:ix+ix_source], x[ix+ix_source:ix+ix_target*2]))
    return data
    

In [0]:
dummy_data = generate_dummy_data()

In [0]:
shuffle(dummy_data)

## Data Preprocess

In [0]:
#vocabulary preparation
vocab = []
for inp, out in data_tr:
    vocab+=[w for w in inp]
    vocab+=[w for w in out]
vocab = list(set(vocab))
vocab.insert(0, '<PAD>')
vocab.append('<UNK>')
print(vocab)

In [0]:
w2id = {w:i for i, w in enumerate(vocab)}
id2w = {w:i for i, w in w2id.items()}

## Auxiliary Functions

In [0]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0  and epoch>0:
            nb_samples = 1
            data_t = sample(data_tr, nb_samples)
            data_test = []
            for inp, out in data_t:
                ind_enc_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in inp]
                ind_dec_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in out]
                data_test.append((ind_enc_in,ind_dec_in))

            params = {
                'max_encoder_len': maxlen_source+2,
                'max_decoder_len': maxlen_source+2,
                'target_len': len(vocab)
                }

            encoder_input_data = np.zeros(shape=(nb_samples, params['max_encoder_len'], params['target_len']))    
            decoder_input_data = np.zeros(shape=(nb_samples, params['max_decoder_len'], params['target_len']))

            for i, (ei, di) in enumerate(data_test):
                for j, idx in enumerate(ei):
                    encoder_input_data[i, j, idx] = 1
                for j, idx_di in enumerate(di):
                    decoder_input_data[i, j, idx_di] = 1

            result = self.model.predict([encoder_input_data, decoder_input_data])
            for r, original in zip(result, data_t):
                original_sentence = original[0]
                idx = np.argmax(r, axis=1)
                print(idx)
                repr_out = []
                for ix in idx:
                    token = id2w[ix]
                    if token == '<EOS>':
                        break
                    else:
                        repr_out.append(token)
                #print('   '*40, end='\r')
                print('Test Sample epoch({}): {} ====> {}'.format(epoch, original_sentence, " ".join(repr_out)))

In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % 3 == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g--', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b--', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

## Architecture definition

In [0]:
class Seq2Seq:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
    
    def compile_basic_seq2seq(self, params={}):
        
        
        return "model"
        
    def train(self, model, data, params={}):
        
        callbacks = self._get_callbacks()
        if 'shuffle' in params and params['shuffle']:
            shuffle(data)
        
        encoder_input_data = np.zeros(shape=(len(data), train_params['max_encoder_len'], train_params['target_len']))    
        decoder_input_data = np.zeros(shape=(len(data), train_params['max_decoder_len'], train_params['target_len']))
        decoder_target_data = np.zeros(shape=(len(data), train_params['max_decoder_len'], train_params['target_len']))
        for i, (ei, di,dt) in enumerate(data):
            for j, idx in enumerate(ei):
                encoder_input_data[i, j, idx] = 1
            for j, idx_di in enumerate(di):
                decoder_input_data[i, j, idx_di] = 1
            for j, idx_dt in enumerate(dt):
                decoder_target_data[i, j, idx_dt] = 1       
                
        model.fit("definir fit")
            
    def predict(self, model, data, params={}):
        
        encoder_input_data = np.zeros(shape=(len(data), params['max_encoder_len'], params['target_len']))    
        decoder_input_data = np.zeros(shape=(len(data), params['max_decoder_len'], params['target_len']))
        
        for i, data_in in enumerate(data):
            if len(data_in) == 3:
                (ei, di, _) = data_in
            else:
                (ei, di) = data_in
            for j, idx in enumerate(ei):
                encoder_input_data[i, j, idx] = 1
            for j, idx_di in enumerate(di):
                decoder_input_data[i, j, idx_di] = 1
                
        return model.predict([encoder_input_data, decoder_input_data])
    
    def load(self, model_path='seq2seq.h5'):
        return load_model(model_path)
    
    def _get_callbacks(self, model_path='seq2seq.h5'):
        es = EarlyStopping(monitor='loss', patience=20, mode='auto', verbose=1)
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only=True, save_weights_only=False, period=2)
        st = Sampletest()
        # hd = HistoryDisplay()
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2,
              patience=5, min_lr=0.0001, verbose=1)
        return [st, save_best, rlr]

## Compile model definition

## Seq2Seq Train

## Predict